# Project - Price forecasting for volatile digital assets

Sharon's log/short description here

# Dependencies

In [2]:
import yfinance as yf
import pandas as pd
import psycopg2


# Data Pipeline

In [8]:
# original function with issue with function returning only one cryptocurrency
# create a function to establish a connection with the Api and retrieve data
def retrieve_data(symbols, start_date, end_date):
    for each_symbols in symbols:
        crypto_data = yf.Ticker(f"{each_symbols}-USD").history(start = start_date, end = end_date, interval = "1d")
        crypto_data_df = pd.DataFrame(crypto_data)
        crypto_data_df["Symbols"] = each_symbols
        crypto_data_df.columns.values[6] = 'Stock_Splits'

    return crypto_data_df


In [3]:
# Fixed issue with function returning only one cryptocurrency
# create a function to establish a connection with the Api and retrieve data
def fixed_retrieve_data(symbols, start_date, end_date):
    data_holder = pd.DataFrame() # Create an empty DataFrame to store all data

    for each_symbols in symbols:
        crypto_data = yf.Ticker(f"{each_symbols}-USD").history(start = start_date, end = end_date, interval = "1d")
        crypto_data_df = pd.DataFrame(crypto_data)
        crypto_data_df["Symbols"] = each_symbols
        crypto_data_df.columns.values[6] = 'Stock_Splits'
        all_dataframes = [data_holder, crypto_data_df]
        data_holder = pd.concat(all_dataframes)

    return data_holder



In [4]:
symbols = ["BTC", "ETH", "USDT"]
start_date = "2015-01-01"
end_date = "2023-12-15"
# table_name = "crypto_table"

view_all = fixed_retrieve_data(symbols, start_date, end_date)
view_all

,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Symbols
Date,,,,,,,,
2015-01-01,320.434998,320.434998,314.002991,314.248993,8036550,0,0,BTC
2015-01-02,314.079010,315.838989,313.565002,315.032013,7860650,0,0,BTC
2015-01-03,314.846008,315.149994,281.082001,281.082001,33054400,0,0,BTC
2015-01-04,281.145996,287.230011,257.612000,264.195007,55629100,0,0,BTC
2015-01-05,265.084015,278.341003,265.084015,274.473999,43962800,0,0,BTC
...,...,...,...,...,...,...,...,...
2023-12-11,1.000072,1.000773,0.998850,0.999547,73473005079,0,0,USDT
2023-12-12,0.999582,1.000987,0.999030,0.999786,50960203250,0,0,USDT
2023-12-13,0.999721,1.000719,0.999246,1.000191,52038965688,0,0,USDT


In [9]:
symbols = ["BTC", "ETH", "USDT"]
start_date = "2015-01-01"
end_date = "2023-12-15"
table_name = "crypto_table"

data = retrieve_data(symbols, start_date, end_date)
# new_data = data.rename(columns={'Stock Split': 'Stock_Splits'})
# If you don't use inplace=True you have to assign it to a new dataframe
# data.columns.values[6] = 'Stock_Splits' # resolve KeyError: 'Stock_Splits'
new_data = data.rename_axis('Date').reset_index() # resolve KeyError: 'Symbols'
# new_data.columns.values[8] = 'Symbols' # resolve KeyError: 'Symbols'
new_data

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Symbols
0,2017-11-09,1.010870,1.013270,0.996515,1.008180,358188000,0,0,USDT
1,2017-11-10,1.006500,1.024230,0.995486,1.006010,756446016,0,0,USDT
2,2017-11-11,1.005980,1.026210,0.995799,1.008990,746227968,0,0,USDT
3,2017-11-12,1.006020,1.105910,0.967601,1.012470,1466060032,0,0,USDT
4,2017-11-13,1.004480,1.029290,0.975103,1.009350,767884032,0,0,USDT
...,...,...,...,...,...,...,...,...,...
2223,2023-12-11,1.000072,1.000773,0.998850,0.999547,73473005079,0,0,USDT
2224,2023-12-12,0.999582,1.000987,0.999030,0.999786,50960203250,0,0,USDT
2225,2023-12-13,0.999721,1.000719,0.999246,1.000191,52038965688,0,0,USDT
2226,2023-12-14,1.000275,1.000986,0.999786,1.000155,50697278985,0,0,USDT


In [21]:
# Create a function to store a copy in SQL -
# updated w/Abel's database details

def sql_data_storage(data, table_name):
    try:
        # Create connection to the SQL database
        connection = psycopg2.connect(
            dbname = 'crypto_database',
            user = 'abelakeni',
            password = 'postgresdb',
            host = '34.30.85.80',
            port = '5432'   
        )
        # Create a cursor 
        cursor = connection.cursor()
      
        # Create a table
        table_query = f"CREATE TABLE IF NOT EXISTS {table_name} (Date DATE, Open FLOAT, High FLOAT, Low FLOAT, Close FLOAT, Volume FLOAT, Dividends FLOAT , Stock_Splits FLOAT, Symbols VARCHAR(225)) "
        cursor.execute(table_query)
       
        # Insert data into the table
        for index, row in data.iterrows():
            insert_query = f"INSERT INTO {table_name} (Date, Open, High, Low, Close, Volume, Dividends, Stock_Splits, Symbols) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s)"
            values = (row["Date"], row["Open"], row["High"], row["Low"], row["Close"], row["Volume"], row["Dividends"], row["Stock_Splits"], row["Symbols"])
            cursor.execute(insert_query, values)
       
        # Commit the data into the connection
        connection.commit()
        
        # Success note/ Inform user
        print("Your data has been inserted successfully into the SQL database")
        
    except psycopg2.Error as error:
        print(f" an error has occurred : {error}")
        
    finally:
        if "connection" in locals() and connection is not None:
            cursor.close()
            connection.close()
            print("Your connection is closed")



In [28]:
sql_data_storage(new_data, table_name)

Your data has been inserted successfully into the SQL database
Your connection is closed


In [3]:
# create a function to Connect to an SQL database to retrieve Data
def retrieve_sql_data(table_name):
    try:
        # Create a connection with the database
        connection = psycopg2.connect(
            dbname = 'crypto_database',
            user = 'sharon_anya',
            password = 'datapipelines',
            host = '35.193.208.235', 
            port = '5432'   
        )
        # Create a cursor
        cursor = connection.cursor()
        
        # Retrieve Data from the table
        table_query = f"SELECT * FROM {table_name}"
        cursor.execute(table_query)
        result = cursor.fetchall()
        
        # Convert the data from the SQL database to a dataframe
        column_name = [header[0] for header in cursor.description] # what other items are held in the cursor.description 
        crypto_data_df = pd.DataFrame(result, columns= column_name) 
        
    except psycopg2.Error as error:
        print(f"You have encountered an error: {error}")
    finally:
        if "connection" in locals() and connection is not None:
            cursor.close()
            connection.close()
            print("Your connection is closed")
    return crypto_data_df



In [6]:
# Conditional statement to switch to an SQL database
# if API data is null get data from the SQL database
symbols = ["BTC", "ETH", "USDT"]
start_date = "2015-01-01"
end_date = "2023-11-30"
table_name = "crypto_table"

store_data = retrieve_data(symbols, start_date, end_date)
if store_data is None:
    store_data = retrieve_sql_data(table_name)

if store_data is None:
    store_data = pd.read_csv("data/soligencecryptodata_withdate.csv")

store_data.head()



# Retrieve data from a csv file

# Store data in a pandas dataframe

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2017-11-09 00:00:00+00:00,1.01087,1.01327,0.996515,1.00818,358188000,0.0,0.0,USDT
2017-11-10 00:00:00+00:00,1.00650,1.02423,0.995486,1.00601,756446016,0.0,0.0,USDT
2017-11-11 00:00:00+00:00,1.00598,1.02621,0.995799,1.00899,746227968,0.0,0.0,USDT
2017-11-12 00:00:00+00:00,1.00602,1.10591,0.967601,1.01247,1466060032,0.0,0.0,USDT
2017-11-13 00:00:00+00:00,1.00448,1.02929,0.975103,1.00935,767884032,0.0,0.0,USDT


In [7]:
# NO INTERNET
symbols = ["BTC", "ETH", "USDT"]
start_date = "2015-01-01"
end_date = "2023-11-30"
table_name = "crypto_table"

store_data_2 = retrieve_data(symbols, start_date, end_date)
if store_data_2 is None:
    store_data_2 = retrieve_sql_data(table_name)
if store_data_2 is None:
    store_data_2 = pd.read_csv("data/soligencecryptodata_withdate.csv") 
store_data_2.head()


,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2017-11-09 00:00:00+00:00,1.01087,1.01327,0.996515,1.00818,358188000,0.0,0.0,USDT
2017-11-10 00:00:00+00:00,1.00650,1.02423,0.995486,1.00601,756446016,0.0,0.0,USDT
2017-11-11 00:00:00+00:00,1.00598,1.02621,0.995799,1.00899,746227968,0.0,0.0,USDT
2017-11-12 00:00:00+00:00,1.00602,1.10591,0.967601,1.01247,1466060032,0.0,0.0,USDT
2017-11-13 00:00:00+00:00,1.00448,1.02929,0.975103,1.00935,767884032,0.0,0.0,USDT


In [4]:
symbols = ["BTC", "ETH", "USDT"]
start_date = "2015-01-01"
end_date = "2023-11-30"
test_data = retrieve_data(symbols, start_date, end_date)
test_data.head(5)

,Open,High,Low,Close,Volume,Dividends,Stock Splits,Symbol
Date,,,,,,,,
2017-11-09 00:00:00+00:00,1.01087,1.01327,0.996515,1.00818,358188000,0.0,0.0,USDT
2017-11-10 00:00:00+00:00,1.00650,1.02423,0.995486,1.00601,756446016,0.0,0.0,USDT
2017-11-11 00:00:00+00:00,1.00598,1.02621,0.995799,1.00899,746227968,0.0,0.0,USDT
2017-11-12 00:00:00+00:00,1.00602,1.10591,0.967601,1.01247,1466060032,0.0,0.0,USDT
2017-11-13 00:00:00+00:00,1.00448,1.02929,0.975103,1.00935,767884032,0.0,0.0,USDT


In [ ]:
sql_data_storage(test_data,"crypto_table")


In [ ]:
# Date, Open, High, Low, Close, Volume, Dividends, Stock Splits, Symbol

# Data Quality Testing

In [10]:
# counting occurences of a particular entry
# checking for inconsistent entries and volume of occurrence (e.g. JAN, jan, JAN or credit_meter, credit, cred etc)

"""
1. Counts total occurrence of unique entries in a particular column.
2. Reset index transforms it into a dataframe with two columns: unique entry & corresponding counts
3. Rename each columns to legible, intuitive names
4. Get the total for the counts, create a dictionary with 'Total' and corresponding 'Total_Count'
5. Append the dictionary in #4 above to the entry_count dataframe
"""
crypto_type_count = new_data['Symbols'].value_counts().reset_index()
crypto_type_count.columns = ['crypto_symbol','count_per_symbol' ]
total_count = crypto_type_count['count_per_symbol'].sum()
total_row = {'crypto_symbol': 'Total',
             'count_per_symbol': total_count}


# crypto_type_count = crypto_type_count.append(total_row, ignore_index=True) # This method produced Error xxx: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.

total_row = pd.DataFrame([total_row]) # in converting a dictionary to a dataframe, enclose the dictionary in a square bracket.
all_dataframes = [crypto_type_count, total_row]

crypto_type_count = pd.concat(all_dataframes, ignore_index=True)

# Show summary dataframe
crypto_type_count
# total_row

# rename

,crypto_symbol,count_per_symbol
0,USDT,2228
1,Total,2228


In [11]:
# Alternative method - using groupby
"""
Dataframe can be sorted in ascending or descending order
sorted_df = crypto_type_count2.sort_values(by='count', ascending=True)

"""
# You can also count unique occurrence of a pair of columns by including them in the list for .groupby([])
crypto_type_count2 = new_data.groupby(['Symbols']).size().reset_index(name='count') #reset index so the column becomes a proper column and not an index
crypto_type_count2



,Symbols,count
0,USDT,2228


In [12]:
crypto_type_count3 = view_all.groupby(view_all.index.year).size() #reset index so the column becomes a proper column and not an index
crypto_type_count3


Date
2015     365
2016     366
2017     471
2018    1095
2019    1095
2020    1098
2021    1095
2022    1095
2023    1047
dtype: int64

In [ ]:
# Revisit code from InsightsPaper_undelivered
# There is an interesting code mapping two dataframes, and fillling NA  towards the end




In [15]:
# check duplicates in a specific column
crypto_duplicates = new_data['Symbols'].duplicated()

"""

columns_to_check = ['Date']
Alternatively this code below also works
crypto_specific_columns = new_data.duplicated(subset=columns_to_check)
rows_with_duplicates = new_data[crypto_specific_columns]
print(rows_with_duplicates)

"""
# check duplicates in whole dataframe. returns a new dataframe with true of false per row
# crypto_duplicates = new_data.duplicated()

# display only rows with duplicate entries
duplicate_rows = new_data[crypto_duplicates]
# crypto_duplicates
duplicate_rows

,Date,Open,High,Low,Close,Volume,Dividends,Stock_Splits,Symbols
1,2017-11-10,1.006500,1.024230,0.995486,1.006010,756446016,0,0,USDT
2,2017-11-11,1.005980,1.026210,0.995799,1.008990,746227968,0,0,USDT
3,2017-11-12,1.006020,1.105910,0.967601,1.012470,1466060032,0,0,USDT
4,2017-11-13,1.004480,1.029290,0.975103,1.009350,767884032,0,0,USDT
5,2017-11-14,1.005240,1.013430,0.996898,1.006830,429857984,0,0,USDT
...,...,...,...,...,...,...,...,...,...
2223,2023-12-11,1.000072,1.000773,0.998850,0.999547,73473005079,0,0,USDT
2224,2023-12-12,0.999582,1.000987,0.999030,0.999786,50960203250,0,0,USDT
2225,2023-12-13,0.999721,1.000719,0.999246,1.000191,52038965688,0,0,USDT
2226,2023-12-14,1.000275,1.000986,0.999786,1.000155,50697278985,0,0,USDT


In [55]:
# Missing Data - count missing data per column

missing_data = new_data.isna() # the aliase .isnull() can also be used
missing_data_count = missing_data.sum()
missing_data_count

Date            0
Open            0
High            0
Low             0
Close           0
Volume          0
Dividends       0
Stock_Splits    0
Symbols         0
dtype: int64

In [ ]:
# Missing Data - count entire rows with missing data
missing_values = new_data.isna().sum(axis=1)
rows_missing = (missing_values >= 9).sum()
rows_missing

In [70]:
# Missing Data - check for missing entries in time series
inferred_freq = pd.infer_freq(new_data['Date'])
print(f"Inferred frequency: {inferred_freq}")

Inferred frequency: D


In [72]:
# inferred_freq = view_all.index.inferred_freq # use this code if the date is still the index of the data frame

print(f"Inferred frequency: {inferred_freq}")

Inferred frequency: None


# Data Understanding

# Feature Selection

In [1]:
# Domain Knowledge

# Data preprocessing

# Analytical Models

# Model Evaluation

# GUI